In [1]:
import numpy as np
import math
import cv2 
import os
from matplotlib import pyplot as plt
# from google.colab.patches import cv2_imshow

In [2]:
im = cv2.imread('pxn.jpg')
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, 1, 2)

In [3]:
# cv2_imshow(im)

In [4]:
# for cnt in contours:
#   x,y,w,h = cv2.boundingRect(cnt)
#   cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),2)
#   crop_img = im[y:y+h,x:x+w]
#   cv2_imshow(crop_img)
  

In [ ]:
im = cv2.imread('./pxn.jpg')
imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, 1, 2)
# cv2_imshow(im)

groups=[]
minRec=63
maxRec=2000
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    if(w*h>minRec and w*h<maxRec):
        groups.append([x,y,w,h])
    
i=0
j=1
max_d= 40
dictGroups={}
a=0
b=0
recs=[]
copys=groups
groupName=0
isNotRecList=False

def distance(rec1,rec2):
    return (((int(rec2[0])-int(rec1[0]))**2+(int(rec2[1])-int(rec1[1]))**2)**0.5)

def filterRectangle(recsTemp,currCopys):
    recsTemp1=[]
    a=0
    b=0
    isNotRecList=False
    while(a<len(recsTemp)):
        while(b<len(currCopys)):
            if(distance(recsTemp[a],currCopys[b])<=max_d):
                recs.append(currCopys[b])
                recsTemp1.append(currCopys[b])
                currCopys.pop(b)
            b+=1
    a+=1
    b=0
    if(len(recsTemp1)==0):
        isNotRecList=True
    if(isNotRecList):
        isNotRecList=False
        return recs
    return filterRectangle(recsTemp1,currCopys)

while(len(copys)>0):
    recs.append(copys[i])
    copys.pop(i)
    while(j<len(copys)):
        if(distance(recs[-1],copys[j])<=max_d):
            recs.append(copys[j])
            copys.pop(j)
        j+=1
    dictGroups[groupName]=recs
    groupName+=1
    filterRectangle(recs,copys)
    recs=[]
    j=1
    print('done clustering')

xPoints = []
yPoints = []
coordinateRecs = []
def calculateHeightNWidth(xMin, xMax, yMin, yMax):
    w2 = int(distance([xMin, yMax], [xMax, yMax]))
    h2 = int(distance([xMin, yMax], [xMin, yMin]))
    return [w2, h2]


def getCoordinateRec(xList, yList):
    xMin = min(xList)
    xMax = max(xList)
    yMin = min(yList)
    yMax = max(yList)
    size = calculateHeightNWidth(xMin, xMax, yMin, yMax)
    coordinateRecs.append([[xMin, yMax], [xMin, yMin], [xMax, yMax], [
                        xMax, yMin], size[0], size[1]])


for i, items in enumerate(dictGroups.values()):
    xPoints=[]
    yPoints=[]
    for item in items:
        xPoints.append(item[0])
        yPoints.append(item[1])
    getCoordinateRec(xPoints, yPoints)
    
for i,item in enumerate(coordinateRecs):
    print(i)
    print(item)

In [ ]:
coordinateRecs[0][4]

In [ ]:
margin=10
for i,image in enumerate(coordinateRecs):
    crop_img=[]
    grey_image=[]
    if(image[0][0]<4*margin):
    crop_img = new_im[image[0][1]-2*margin-image[5]:image[0][1]+4*margin, image[0][0]:image[0][0]+image[4]+5*margin]
    elif(image[0][1]<2*margin+image[5]):
    crop_img = new_im[image[0][1]-2*margin-image[5]:image[0][1]+4*margin, image[0][0]-4*margin:image[0][0]+image[4]+5*margin]
    else:
    crop_img = new_im[image[0][1]-2*margin-image[5]:image[0][1]+4*margin, image[0][0]-4*margin:image[0][0]+image[4]+5*margin]
    grey_image=cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
    resize_image= cv2.resize(grey_image,(280,32))
#   cv2_imshow(resize_image)
#   cv2_imshow(im)
#   cv2.waitKey(0)
    print('--------------------------------------------------------------')
    cv2.imwrite('./img'+str(i)+'.png',resize_image)